In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32,padding = 4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
train_cifar = datasets.CIFAR10(
    root = 'data',
    train = True,
    download = True,
    transform = transform
)
test_cifar = datasets.CIFAR10(
    root = 'data',
    train = False,
    download = True,
    transform = transform
)

In [ ]:
train_loader = DataLoader(train_cifar, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_cifar, batch_size = 64, shuffle = False)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class CNN2(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size = 3, stride =1, padding =2)
    self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, stride =1, padding =2)
    self.conv3 = nn.Conv2d(64, 128, kernel_size = 3, stride =1, padding =2)
    self.fc1 = nn.Linear(3200, 256)
    self.fc2 = nn.Linear(256, 10)
    self.pool = nn.MaxPool2d(2)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    x = self.conv1(x)
    x = nn.ReLU()(x)
    x = self.pool(x)
    #print(x.size())
    x = self.conv2(x)
    x = nn.ReLU()(x)
    x = self.pool(x)
   # print(x.size())
    x = self.conv3(x)
    x = nn.ReLU()(x)
    x = self.pool(x)
    #print(x.size())
    x = x.view(-1, 3200)
    x = self.fc1(x)
    x = nn.ReLU()(x)
    x = self.fc2(x)
    x = self.dropout(x)
    return x

model = CNN2().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

1.0 2.4 1.3 1.2 2.0

In [ ]:
def func1(x):
  return x**2

def func2(x):
  return x/2

y = func1(func2(x))

In [ ]:
(64, 28*28)

In [ ]:
class CIFARCNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,32, kernel_size=3, stride = 1, padding =1)
    self.bn1 = nn.BatchNorm2d(32)

    self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, stride =1, padding =2)
    self.bn2 = nn.BatchNorm2d(64)

    self.conv3 = nn.Conv2d(64, 128, kernel_size = 3, stride =1, padding =2)
    self.bn3 = nn.BatchNorm2d(128)

    self.fc1 = nn.Linear(3200, 256)
    self.fc2 = nn.Linear(256, 10)
    self.pool = nn.MaxPool2d(2)
    self.dropout = nn.Dropout(0.2)

    self.relu = nn.ReLU()
    self.pool = nn.MaxPool2d(2)
  def forward(self,x):
    x = self.pool(self.relu(self.bn1(self.conv1(x))))
    x = self.pool(self.relu(self.bn2(self.conv2(x))))
    x = self.pool(self.relu(self.bn3(self.conv3(x))))
    x = x.view(x.size(0), -1)
    x = self.dropout(self.relu(self.fc1(x)))
    return self.fc2(x)

model = CIFARCNN().to(device)
model.conv3 =
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr = 0.01)
optimizer = optim.Adam(model.parameters(), lr=0.001)





In [ ]:
def evaluate(model, loader):
  model.eval()
  correct = 0
  with torch.no_grad():
    for X, y in loader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      prediction = pred.argmax(dim = 1)
      correct += (prediction == y).sum().item()
  return correct/len(loader.dataset)


def train_one_epoch(model, loader, optimizer, criterion):
    model.train(); run_loss = 0
    for X, y in loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(model(X), y)
        loss.backward(); optimizer.step()
        run_loss += loss.item() * X.size(0)
    return run_loss / len(loader.dataset)

for epoch in range(10):
    loss = train_one_epoch(model, train_loader, optimizer, criterion)
    train_acc = evaluate(model, train_loader)
    val_acc   = evaluate(model, test_loader)
    print(f"Epoch {epoch+1}: loss={loss:.4f} train={train_acc:.3f} val={val_acc:.3f}")

Epoch 1: loss=0.8317 train=0.794 val=0.785
Epoch 2: loss=0.6188 train=0.808 val=0.797


KeyboardInterrupt: 

In [ ]:
from torchvision import models
model = models.resnet18(weights = models.ResNet18_Weights.IMAGENET1K_V1).to(device)
for p in model.parameters():
  p.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)







data
  train
    dogs
      img1
      img2
      img3
    cats
      img1
      img2
      img3


      
  test.csv
  test
    img1

In [ ]:
import zipfile, pathlib, shutil
zip_path = "archive.zip"
extract_to = pathlib.Path("data/catsdogs")

with zipfile.ZipFile(zip_path) as zf:
  zf.extractall(extract_to)

cat 0
dog 1
ship 2
train 3

In [ ]:
(1, 32, 32, 3)

/content/data/catsdogs/train/dogs/dog_0.jpg

---



In [ ]:
import os
from torch.utils.data import Dataset
from PIL import Image

TRAIN_ROOT = '/content/data/catsdogs/train'
TEST_ROOT = '/content/data/catsdogs/test'

class CatsDogsDataset(Dataset):
  def __init__(self, root, transform = None):
    self.root = pathlib.Path(root)
    self.transform = transform
    self.label_map = {'cats': 0, 'dogs': 1}
    self.images = list(self.root.glob('*/*.jpg'))

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_path = self.files[idx]
    img = Image.open(img_path).convert('RGB')
    label_str = img_path.parent.name
    y = self.label_map[label_str]
    if self.transform is not None:
      img = self.transform(img)
    return img, torch.tensor(y, dtype = torch.long)






